Устанавливаем библиотеку для работы с gpu с помощью данной команды:

pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [2]:
import torch
# проверяем увидила ли наша библиотека gpu устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if device == torch.device("cuda"):
    device = 0

cuda


Учитывая ограничение по времени и мощности будем использовать библиотеку YOLO. Это открытая библиотека с уже предообученной моделью, что упростит нам задачу и время, мы будем просто переобучать её на наши данные, а не обучать с нуля (Что заняло бы достаточно времени). Я выбрал модель yoloV8 так как она более современная и подерживает работу не только через терминал, по сравнению с yolov5.

Устанавливаем сопуствующие пакеты к библиотеке.
pip install ultralytics - сама библиотека содержащая все модели yolo

pip install supervision - полезный пакет для взаимодействия с камерой

Устанавливаем Библиотеку scikit-learn, в ней есть полезные методы для машинного обучения, а также функции упрощающие работу.
 
pip install scikit-learn

In [ ]:
import os
# считываем наши файлы в списки изображений и разметки 
images = [os.path.join('all/images', x) for x in os.listdir('all/images')if x.endswith('.jpg')]
labels = [os.path.join('all/images', x) for x in os.listdir('all/images')if x.endswith('.txt')]
# сортируем их
images.sort()
labels.sort()

In [ ]:
from sklearn.model_selection import train_test_split
# производим разделение всех наших данных на Тренировочные и валидационные
train_images, val_images, train_annotations, val_annotations = train_test_split(images, labels, test_size=0.2, random_state=42)
# валидационные данные делим на тестовые и валидационные, чтобы одни использовать для тестирования пока обучается, а вторые для тестирования после обучения
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size=0.5, random_state=42)

In [ ]:
# Создаем папки куда будем переносить наши разбитые на 3 набора данные фотографий
parent = 'all'
dirs = 'images/train', 'images/val', 'images/test', 'labels/train', 'labels/val', 'labels/test'
for dir in dirs:
    path = os.path.join(parent, dir)
    # проверяем существование папки чтобы не создавать если уже есть
    if not os.path.exists(path):
        # создаем папку
        os.makedirs(path)

In [ ]:
import shutil
#Утилитная функция для переноса файлов 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

# Переносим файлы по из папкам учитывая разделение
move_files_to_folder(train_images, 'all/images/train')
move_files_to_folder(val_images, 'all/images/val/')
move_files_to_folder(test_images, 'all/images/test/')
move_files_to_folder(train_annotations, 'all/labels/train/')
move_files_to_folder(val_annotations, 'all/labels/val/')
move_files_to_folder(test_annotations, 'all/labels/test/')

Теперь создаем файл data.yaml.
В этом файле хранится информация путей к нашим данным для обучения и тренировки, а также классы которые у нас есть.

In [3]:
#  импортируем YOLO класс из библиотеки ultralytics
from ultralytics import YOLO

# Создаем новую YOLO модель с нуля
model = YOLO('yolov8n.yaml')
# загружаем претренированную YOLO модель
model = YOLO('yolov8n.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

In [3]:
# Трнируем модель на основе 'data.yaml' файла нашего датасета на 10 эпох
results = model.train(data='data.yaml', epochs=10,  device=device, name='custom')

Ultralytics YOLOv8.0.70  Python-3.10.9 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 Super, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=custom, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False,

In [9]:
# Трнируем модель на основе 'data.yaml' файла нашего датасета на 100 эпох
results = model.train(data='data.yaml', epochs=100,  device=device, name='custom')

New https://pypi.org/project/ultralytics/8.0.71 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.70  Python-3.10.9 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 Super, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=../runs/detect/custom/weights/best.pt, data=data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=custom, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=

In [10]:
# Вычисляем точность модели на проверочном наборе данных
results = model.val()

Ultralytics YOLOv8.0.70  Python-3.10.9 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 Super, 8192MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\Users\igorv\OneDrive\Documents\GitHub\datasets\all\labels\val.cache... 338 images, 113 backgrounds, 6 corrupt: 100%|██████████| 338/338 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [01:21<00:00,  3.71s/it]
                   all        338        548       0.53       0.41       0.43      0.248
                   D00        338        254      0.501      0.297      0.292       0.13
                   D10        338        113       0.53      0.407      0.424      0.216
                   D20        338        103      0.611      0.564      0.584      0.303
                   D40        338         70      0.407      0.157      0.168     0.0659
                Repair        338          8      0.601      0.

В качестве API создаем файл camera.py, содержащий функцию main(). И запускающий детекцию дефектов дорожного покрытия. В режиме реального времени. 

python3 -m pip install pytest-cov